In [1]:
import gzip
import csv
import random
import scipy
import tarfile
import pandas as pd
import numpy as np
from sklearn import linear_model
import sklearn
from collections import defaultdict
import re
from string import digits
import urllib
import scipy.optimize
from collections import defaultdict # Dictionaries with default values
import nltk
import string
from nltk.stem.porter import *
import ast
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('data/yelp_reviews.csv')

In [3]:
data = ((data[['user_id', 'business_id', 'stars_x', 'text', 'date', 'name_x', 'name_y', 'stars_y', 'review_count_y', 'is_open', 'attributes', 'categories', 'hours']])
        .rename(columns={'stars_x': 'review_rating', 'name_x':'user_name', 'name_y': 'restaurant_name', 'review_count_y':'num_reviews', 'stars_y':'restaurant_rating'}))

In [4]:
data['popularity'] = data['restaurant_rating'] / data['num_reviews']

In [5]:
all_categories = defaultdict(int)
def clean_text(text):
    punct = string.punctuation
    text = text.replace("\n", ' ').replace("\t", ' ').lower().strip()
    text = [c for c in text if not (c in punct)]
    text = ''.join(text)
    text = text.strip().replace("  ",' ')
    for i in text.split():
        all_categories[i] += 1
    return text

In [6]:
data['categories'] = data['categories'].astype(str)
data['categories'] = data['categories'].apply(clean_text)

In [7]:
catDf = pd.DataFrame(np.zeros([len(data), len(all_categories)]), columns=all_categories)

In [8]:
## LONG ~5 MIN ##
for i in range(len(data)):
    for j in data.iloc[i]['categories'].split():
        catDf.iloc[i][j] = 1

In [9]:
subData = pd.concat([data[['restaurant_rating', 'num_reviews', 'is_open', 'popularity', 'review_rating']], catDf], axis=1).astype(np.float32)

In [10]:
trainData, testData = sklearn.model_selection.train_test_split(subData)

In [11]:
X_train = np.array(trainData.drop(columns=['review_rating']))
y_train = np.array(trainData['review_rating'])

X_test = np.array(testData.drop(columns=['review_rating']))
y_test = np.array(testData['review_rating'])


In [12]:
model = Ridge(alpha=1.5)
model.fit(X_train, y_train)

C:\Users\Andrew Chin\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: Ill-conditioned matrix (rcond=3.36803e-12): result may not be accurate.
  overwrite_a=True).T


Ridge(alpha=1.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [13]:
preds = model.predict(X_test)
mean_squared_error(preds, y_test)

1.6888301